In [ ]:
!pip install opendatasets split-folders

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import os
import shutil
import splitfolders
import opendatasets as od
from google.colab import drive

# --- 1. ПРОВЕРКА И ПОДГОТОВКА ДАННЫХ ---
DATA_DIR = 'dataset'
RAW_DATA_DIR = 'plantdisease'

if not os.path.exists(os.path.join(DATA_DIR, 'train')):

    # Проверка наличия ключа
    if not os.path.exists('kaggle.json'):
        print(" ОШИБКА: Нет файла kaggle.json! Перетащите его в файлы слева.")
    else:
        # Скачивание
        od.download("https://www.kaggle.com/datasets/emmarex/plantdisease")

        # Поиск скачанной папки (она может быть вложенной)
        input_folder = "plantdisease/PlantVillage"
        if not os.path.exists(input_folder):
            # Ищем папку, если путь другой
            for root, dirs, files in os.walk("."):
                if "Tomato_Healthy" in dirs:
                    input_folder = root
                    break
        splitfolders.ratio(input_folder, output=DATA_DIR, seed=1337, ratio=(0.8, 0.2))

# --- 2. НАСТРОЙКИ СОХРАНЕНИЯ ---
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

PROJECT_PATH = '/content/drive/MyDrive/ML_Project_Plants'
SAVE_PATH = f'{PROJECT_PATH}/my_plant_model.pth'
CLASSES_PATH = f'{PROJECT_PATH}/classes.txt'
os.makedirs(PROJECT_PATH, exist_ok=True)

# --- 3. ОБУЧЕНИЕ ---
def run_training():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Трансформации
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    image_datasets = {x: datasets.ImageFolder(os.path.join(DATA_DIR, x), data_transforms[x])
                      for x in ['train', 'val']}

    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True)
                   for x in ['train', 'val']}

    class_names = image_datasets['train'].classes

    # Модель
    print("Готовим ResNet18...")
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    model.fc = nn.Linear(model.fc.in_features, len(class_names))
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    print("\nSTART LEARNING")

    for epoch in range(5):
        print(f'\nEpoch {epoch+1}/5')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train': model.train()
            else: model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    # Сохранение
    torch.save(model.state_dict(), SAVE_PATH)
    with open(CLASSES_PATH, "w") as f:
        f.write("\n".join(class_names))

# Запуск
if __name__ == "__main__":
    run_training()


⚠️ Данные не найдены (Colab их удалил). Скачиваю заново...
Dataset URL: https://www.kaggle.com/datasets/emmarex/plantdisease


100%|██████████| 658M/658M [00:05<00:00, 117MB/s]



✅ Исходники найдены в: plantdisease/PlantVillage
Разбиваем на train/val...


Copying files: 20639 files [00:03, 6268.99 files/s]


✅ Данные восстановлены!
⚙️ Устройство: cuda
📂 Загружаю данные в память...
✅ Найдено классов: 15
🧠 Готовим ResNet18...
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:01<00:00, 44.7MB/s]



🚀 ЗАПУСК ОБУЧЕНИЯ (5 эпох)...

Эпоха 1/5
----------
train Loss: 0.5086 Acc: 0.8616
val Loss: 0.0985 Acc: 0.9741

Эпоха 2/5
----------
train Loss: 0.0976 Acc: 0.9735
val Loss: 0.0550 Acc: 0.9855

Эпоха 3/5
----------
train Loss: 0.0550 Acc: 0.9865
val Loss: 0.0410 Acc: 0.9886

Эпоха 4/5
----------
train Loss: 0.0383 Acc: 0.9907
val Loss: 0.0361 Acc: 0.9903

Эпоха 5/5
----------
train Loss: 0.0259 Acc: 0.9945
val Loss: 0.0292 Acc: 0.9927

💾 Сохраняю модель...
Готово! Файлы сохранены в: /content/drive/MyDrive/ML_Project_Plants
